In [1]:
# # Install tensorflow, instruction below should do that. Be sure that you are in your intended virtual environment
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### I have used content from [Link](https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/classification_iris_data_with_keras.ipynb#scrollTo=2xnX-NTsFQQa)

to understand how to use TPU to run tf keras models

In [18]:
import numpy as np
import tensorflow as tf
import os

In [17]:
print(tf.__version__)
import distutils
if distutils.version.LooseVersion(tf.__version__) < '1.14':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/classification_iris_data_with_keras.ipynb')

2.12.0


<ipython-input-17-821088d430d8>:3: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(tf.__version__) < '1.14':


In [19]:
use_tpu = True

if use_tpu:
    assert "COLAB_TPU_ADDR" in os.environ, "Missing TPU; did you request a TPU in Notebook Settings?"

if "COLAB_TPU_ADDR" in os.environ:
  TF_MASTER = "grpc://{}".format(os.environ["COLAB_TPU_ADDR"])
else:
  TF_MASTER=""

In [20]:
# TPU address
tpu_address = TF_MASTER

In [21]:
tpu_address

'grpc://10.80.220.34:8470'

In [3]:
# Useful pieces
Tokenizer = tf.keras.preprocessing.text.Tokenizer
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences
Sequential = tf.keras.models.Sequential
Embedding = tf.keras.layers.Embedding
SimpleRNN = tf.keras.layers.SimpleRNN
Dense = tf.keras.layers.Dense
LSTM = tf.keras.layers.LSTM
Dropout = tf.keras.layers.Dropout

In [4]:
# Load in data from data.py
from data import text_data_arr

In [5]:
# Tokenize the text at the character level
tokenizer = Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=True,
    oov_token=None,
    analyzer=None
)

In [41]:
# Convert text to sequences (use the tokenizer)
tokenizer.fit_on_texts(text_data_arr)
sequence = tokenizer.texts_to_sequences(text_data_arr)[0] # change for Q1

In [42]:
# Prepare input and output (target) sequences
input_sequences = []

output_sequences = []
sequence_length = 100

curr_seq_len = len(sequence)
start = 0
while start + sequence_length < curr_seq_len:
  end = start + sequence_length
  input_sequences.append(sequence[start: end])
  output_sequences.append(sequence[end])
  start += 1

input_sequences = tf.convert_to_tensor(pad_sequences(input_sequences, maxlen=sequence_length))
output_sequences = tf.convert_to_tensor(output_sequences, dtype=tf.int32)

In [43]:
assert isinstance(input_sequences, tf.Tensor)
assert isinstance(output_sequences, tf.Tensor)
assert len(input_sequences) == len(output_sequences)

In [44]:
len(input_sequences)

876

In [45]:
# set vocab_size based on tokenizer
vocab_size = len(tokenizer.word_index) + 1 # +1 for padding

In [46]:
# Define the model architecture - 4 layers as described below
def create_model():
  return Sequential([
      # Embedding layer - dependent on vocab_size and sequence_length
      Embedding(input_length=sequence_length, input_dim=vocab_size, output_dim=32),
      # LSTM Layer - set size to 128, return sequences, and dropout and recurrent_dropout to .2,
      LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2),
      # LSTM Layer - set size to 128, dropout and recurrent_dropout to .2,
      LSTM(units=128, dropout=0.2, recurrent_dropout=0.2),
      # Dense Layer - dependent on vocab_size, activation is softmax
      Dense(units=vocab_size, activation="softmax"),
  ])

In [47]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

In [48]:
with strategy.scope():
  model = create_model()
  model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 32)           1280      
                                                                 
 lstm_10 (LSTM)              (None, 100, 128)          82432     
                                                                 
 lstm_11 (LSTM)              (None, 128)               131584    
                                                                 
 dense_5 (Dense)             (None, 40)                5160      
                                                                 
Total params: 220,456
Trainable params: 220,456
Non-trainable params: 0
_________________________________________________________________


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #
=================================================================
 embedding (Embedding)       (None, 100, 32)           1280

 lstm (LSTM)                 (None, 100, 128)          82432

 lstm_1 (LSTM)               (None, 128)               131584

 dense (Dense)               (None, 40)                5160

=================================================================
Total params: 220,456
Trainable params: 220,456
Non-trainable params: 0
_________________________________________________________________

In [49]:
# Train the model
epochs = 100
batch_size = 32

model.fit(input_sequences, output_sequences, epochs=epochs, batch_size=batch_size)

Epoch 1/100
28/28 [==============================] - 19s 246ms/step - loss: 3.2967 - accuracy: 0.1678
Epoch 2/100
28/28 [==============================] - 1s 38ms/step - loss: 3.0100 - accuracy: 0.1895
Epoch 3/100
28/28 [==============================] - 1s 38ms/step - loss: 2.9848 - accuracy: 0.1895
Epoch 4/100
28/28 [==============================] - 1s 37ms/step - loss: 2.9737 - accuracy: 0.1895
Epoch 5/100
28/28 [==============================] - 1s 37ms/step - loss: 2.9758 - accuracy: 0.1895
Epoch 6/100
28/28 [==============================] - 1s 38ms/step - loss: 2.9736 - accuracy: 0.1895
Epoch 7/100
28/28 [==============================] - 1s 37ms/step - loss: 2.9665 - accuracy: 0.1895
Epoch 8/100
28/28 [==============================] - 1s 38ms/step - loss: 2.9632 - accuracy: 0.1895
Epoch 9/100
28/28 [==============================] - 1s 37ms/step - loss: 2.9381 - accuracy: 0.1895
Epoch 10/100
28/28 [==============================] - 1s 39ms/step - loss: 2.8820 - accuracy: 0.21

In [50]:
model.save_weights("model_weights.h5")

In [51]:
# Genereate Text
def generate_text(seed_text, model, tokenizer, sequence_length, num_chars_to_generate):
    generated_text = seed_text
    for _ in range(num_chars_to_generate):
        # create a token_list from your generated text, be sure to pad the sequences
        token_list = tokenizer.texts_to_sequences([generated_text])
        token_list = pad_sequences(token_list, maxlen=sequence_length)

        # predict the probabilities of each token
        predicted_probs = model.predict(token_list)
        predicted_probs = predicted_probs[0]

        # get the predicted token by finding the index of with the highest probability
        predicted_token_index = tf.math.argmax(predicted_probs)
        output_letter = ""
        for letter, index in tokenizer.word_index.items():
            # get the letter which corresponds to the predicted_token
            if index == predicted_token_index:
              output_letter = letter
              break
        generated_text += output_letter
    return generated_text

In [52]:
seed_text = "John: How are you, Sarah?"

In [53]:
generated_text = generate_text(seed_text, model, tokenizer, sequence_length, num_chars_to_generate=100)
print(generated_text)

1/1 [==============================] - 1s 547ms/step
John: How are you, Sarah? take care! been up to anything interesting lately? mike: actually, i went on a hiking trip last wee
